In [1]:
import joblib
import pandas as pd
import numpy as np

# Cargar modelo entrenado
try:
    model = joblib.load("modelo_xgboost_f1opt.joblib")
    print("Modelo XGBoost cargado correctamente.")
except FileNotFoundError:
    raise FileNotFoundError("No se encontró el archivo del modelo: 'modelo_xgboost_f1opt.joblib'.")

# Cargar umbral F1-óptimo
try:
    with open("umbral_f1.txt", "r") as f:
        best_threshold = float(f.read().strip())
    print(f"Umbral F1-óptimo cargado: {best_threshold:.4f}")
except FileNotFoundError:
    best_threshold = 0.5
    print("No se encontró el archivo 'umbral_f1.txt'. Se usará el valor por defecto de 0.5 (o edite manualmente).")

# Cargar variables seleccionadas
with open("variables_seleccionadas.pkl", "rb") as f:
    selected_features = joblib.load(f)

print(f"Variables cargadas correctamente: {selected_features}")

Modelo XGBoost cargado correctamente.
Umbral F1-óptimo cargado: 0.3413
Variables cargadas correctamente: ['age', 'high-school_average', 'conditioned_career', 'civil_family_burden', 'academic_progress_performance', 'economic_burden', 'public transport', 'tuition_fee', 'lifestyle_free_time', 'educational_capital_father_mother', 'work-career_relationship', 'sporadic_work_shift', 'non-motorized_transport', 'economic constraint', 'physical_teaching_resources', 'evening_work_shift', 'weekend_work_shift', 'personal_limitation', 'night_shift_work', 'career_satisfaction', 'university_transfer_time', 'public_institution', 'economic_restriction', 'financial_independence', 'distance_campus', 'private_institution', 'goods_services_housing', 'vocational_influence', 'mixed_work_shift', 'intensity_tobacco_alcohol_consumption', 'work_field_private_entrepreneur', 'schooled_institution', 'techniques_study_organization', 'public_institutional_labor_scope', 'educational_parental_assessment', 'institutional

In [2]:
import pandas as pd
import numpy as np

# Cargar dataset preparado para predicción (despues del preprocesamiento)
try:
    df = pd.read_csv('data/variables.csv')  # Cambiar la ruta si es necesario
    print("Dataset cargado exitosamente.")
except FileNotFoundError:
    raise FileNotFoundError("El dataset 'variables.csv' no fue encontrado")

# Filtrar cohortes 2023 y 2024
df_pred = df[df['Año'].isin([2023, 2024])].copy()

# Guardar identificadores relevantes
ids = df_pred[['EMPLID', 'Año']].copy()

# Eliminar las mismas columnas numéricas usadas en entrenamiento
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
drop_unwanted = [
    'ETNIA', 'ADEUDO', 'PUNTAJE_ADM', 'CAMBIO_PROGRAMA', 'CARRERA', 'Año', 'CATEGORIA_CARRERA','BACH_PROMEDIO','EDAD',
    'SEMESTRE PLAN', 'ÚLTIMO PERIODO CURSADO', 'SEMESTRE ABANDONO', 'DESERCION', 'NIVEL_CARRERA',  
    'EMPLID', 'Latitud', 'Longitud'  
]
features = [c for c in numeric_cols if c not in drop_unwanted]

# Extraer las variables predictoras
X_pred = df_pred[features]

# Verificar número de características
print(f"Número de características en X_pred: {X_pred.shape[1]}")
print("Características:", features)

Dataset cargado exitosamente.
Número de características en X_pred: 68
Características: ['economic_burden', 'financial_independence', 'tuition_fee', 'resources_academic_activities', 'accessibility', 'job_opportunity', 'economic_restriction', 'economic_perception_itson_parents', 'perception_economic_terms', 'economic constraint', 'property_type_housing', 'housing_company', 'passive teaching', 'active-critical-learning-practices', 'teaching_performance', 'cultural_activities', 'services_academic_activities', 'schooled_institution', 'previous_studies_open-line', 'public_institution', 'private_institution', 'prestige_quality', 'academic_progress_performance', 'habits_study_participation', 'physical_teaching_resources', 'digital_teaching_resources', 'techniques_study_organization', 'academic_information_received', 'areas_reinforcement', 'academic_limitation', 'health_condition_special_needs', 'practice_sport', 'personal_limitation', 'intensity_tobacco_alcohol_consumption', 'civil_family_burd

In [3]:
# 3. Aplicar modelo

# Asegurar escalado igual al entrenamiento (modelo incluye scaler)
y_proba = model.predict_proba(X_pred)[:, 1]  
y_pred = (y_proba >= best_threshold).astype(int)  

# Agregar resultados al dataframe original
df_pred = df_pred.copy()
df_pred["probabilidad_desercion"] = y_proba
df_pred["riesgo_predicho"] = y_pred

# Verificar resultados
print(f"\nTotal de estudiantes evaluados: {df_pred.shape[0]}")

print("\nConteo de etiquetas predichas (riesgo de deserción):")
print(pd.Series(y_pred).value_counts().rename(index={0: 'No deserta', 1: 'Deserta'}))

print("\nResumen de probabilidades de deserción:")
print(pd.Series(y_proba).describe())


Total de estudiantes evaluados: 4682

Conteo de etiquetas predichas (riesgo de deserción):
Deserta       2765
No deserta    1917
Name: count, dtype: int64

Resumen de probabilidades de deserción:
count    4682.000000
mean        0.410009
std         0.188484
min         0.050123
25%         0.254541
50%         0.391921
75%         0.552266
max         0.939810
dtype: float64


In [4]:
# Mostrar como resultados

import joblib

# Cargar variables seleccionadas automáticamente
selected_features = joblib.load("variables_seleccionadas.pkl")

# Agregar variables clave adicionales
columnas_salida = (
    ["EMPLID", "Año", "EDAD", "PROMEDIO_BACH", "CARRERA", "AREA_CARRERA", "CAMBIO_PROGRAMA", 
     "CATEGORIA_CARRERA", "NIVEL_CARRERA", 
     "probabilidad_desercion", "riesgo_predicho"]
    + selected_features
)

# Filtrar solo columnas que existan (por si algunas no están presentes)
columnas_salida = [col for col in columnas_salida if col in df_pred.columns]

# Mostrar muestra de los datos
print(df_pred[["Año", "gender", "CARRERA", "AREA_CARRERA", "riesgo_predicho", "probabilidad_desercion"]].head())

# (Opcional) Exportar a CSV
df_pred[columnas_salida].to_csv("resultados_desercion_filtrados.csv", index=False)

        Año  gender CARRERA                    AREA_CARRERA  riesgo_predicho  \
34250  2023     0.0     LEI  Social Sciences and Humanities                1   
34251  2023     1.0   LDCFD  Social Sciences and Humanities                1   
34252  2023     1.0     ARQ      Engineering and Technology                0   
34253  2023     1.0    LAET            Business & Economics                0   
34254  2023     0.0      LA            Business & Economics                0   

       probabilidad_desercion  
34250                0.670592  
34251                0.700410  
34252                0.249835  
34253                0.153251  
34254                0.142207  


In [8]:
# Exportar resultados

# Guardar en CSV
df_pred[columnas_salida].to_csv("riesgo_desercion_2023_2024.csv", index=False)
print("Archivo generado: 'riesgo_desercion_2023_2024.csv'")

Archivo generado: 'riesgo_desercion_2023_2024.csv'


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=24fa97d8-6b94-4f62-a2dc-97416a953ae1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>